In [80]:
 import time                     #importing libraries

## path finding for conflict based search for 2 robots

In [81]:
def pathfinding(maze1, maze2, sp1, sp2):
    p1a = astar(maze1, sp1[0], sp1[1]) # Robot i to Pi
    p1b = astar(maze1, sp1[1], sp1[2]) # pi to Ei
    p1c = astar(maze1, sp1[2], sp1[3])   # Ei to Di
    
    p1 = p1a[:-1]+ p1b[:-1] +p1c # tracing whole path of Robot i
    
    # for path 2
    p2a = astar(maze2, sp2[0], sp2[1]) # Robot j to Pj
    p2b = astar(maze2, sp2[1], sp2[2]) # pj to Ej
    p2c = astar(maze2, sp2[2], sp2[3])   # Ej to Dj
    
    p2 = p2a[:-1]+ p2b[:-1] +p2c # tracing whole path of Robot j
    
    conflict = [] # array will contain conflict nodes of 2 robots
    
    #checking conflict nodes
    for i in range (min(len(p1), len(p2))):
        if (p1[i]==p2[i]):
            conflict.append(p1[i])
            break
    if len(conflict):
        maze1[conflict[0][0]][conflict[0][1]] = 0  #changing conflict node location in maze as obstacle for robot i
        previous_p1, previous_p2, e = pathfinding(maze1, maze2, sp1, sp2) # recursively calculating non conflict best paths
        p1=  previous_p1
        p2 = previous_p2
    
    return p1, p2, maze1  # returning non conflicting nodes of path 
        
            

## Path finding for conflict based search for 3 robots

In [82]:
# function for CBS
def pathfinding_Robots(maze1, maze2, maze3, sp1, sp2, sp3):
    previous_p1, previous_p2, e = pathfinding(maze1, maze2, sp1, sp2) # getting non conflict paths of R1 and R2 by CB search as previous_p1 and previous_p2 respectively
    p1=  previous_p1  # non conflict path of R1 with R2
    p2 = previous_p2  # Non conflict path of R2 with R1
    previous_p3, previous_p2b, maze3b = pathfinding(maze3, maze2, sp3, sp2)  # getting non conflict paths of R3 and R2 by CB search as previous_p3 and previous_p2b respectively
    p3 = previous_p3  # Non conflict path of R3 with R2
    previous_p3b, previous_p1b, e= pathfinding(maze3b, maze1, sp3, sp1) # getting non conflict paths of R3 and R1 by CB search as previous_p3b and previous_p1b respectively
    p3b = previous_p3 # Non conflict path of R3 with R1 and R2.
        
    return p1, p2, p3b  # all non conflict paths of R1, R2 and R3 respectively.

## Node Class

In [83]:
class Node():                  
     #A node class for A* Pathfinding
        
    def __init__(self, prent=None, location=None):
        self.prent = prent
        self.location = location

        self.g = 0  # defining initial path cost (g)
        self.h = 0  # defining initial heuristic path cost (h)
        self.f = 0  # defining initial total cost (f = g + h)
    
    #checking current node is equal to other node
    def __eq__(self, othr):
        return self.location == othr.location


## Function for A* algorithm

In [84]:
def astar(maze, start, end):  #Returns a list of tuples as a path from the given start to the given end in the given maze.

    
    # Create start and end node
    start_nd = Node(None, start)
    start_nd.g = start_nd.h = start_nd.f = 0
    end_nd = Node(None, end)
    end_nd.g = end_nd.h = end_nd.f = 0

    # Initialize both open and closed list
    open_list = []
    closed_list = []

    # Add the start node
    open_list.append(start_nd)

    # Loop until you find the end
    while len(open_list) > 0:

        # Get the current node
        current_node = open_list[0]
        current_index = 0
        for index, item in enumerate(open_list):
            if item.f < current_node.f:   # getting node with optimised f value
                current_node = item
                current_index = index

        # Pop current node from open list, add to closed list
        open_list.pop(current_index)
        closed_list.append(current_node)

        # Found the goal
        if current_node == end_nd:
            path = []
            current = current_node
            while current is not None:
                path.append(current.location)
                current = current.prent
            return path[::-1] # Returning path from start to goal

        # Generate children
        children = []
        for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0)]: # Adjacent top, right, left and bottom node

            # Get node position
            node_position = (current_node.location[0] + new_position[0], current_node.location[1] + new_position[1])

            # Make sure within range
            if node_position[0] > (len(maze) - 1) or node_position[0] < 0 or node_position[1] > (len(maze[len(maze)-1]) -1) or node_position[1] < 0:
                continue

            # Make sure walkable territory
            if maze[node_position[0]][node_position[1]] == 0:
                continue

            # Create new node
            new_node = Node(current_node, node_position)

            # Append children
            children.append(new_node)

        # Loop through children
        for child in children:

            # Child is on the closed list
            for closed_child in closed_list:
                if child == closed_child:
                    continue

            # Create the f, g, and h values for further nodes
            child.g = current_node.g + 1
            child.h = ((child.location[0] - end_nd.location[0]) ** 2) + ((child.location[1] - end_nd.location[1]) ** 2)
            child.f = child.g + child.h

            # Child is already in the open list
            for open_node in open_list:
                if child == open_node and child.g > open_node.g:
                    continue

            # Add the child to the open list
            open_list.append(child)


In [85]:
def main():
    # maze for R1 robot position as 1, P1 as 2, E1 as 3, D1 as 4, obstacle as 0 and all pssible moveable nodes as 1   
    maze1 = [[1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 0, 1, 1, 1],
            [1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1],
            [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1 ],
            [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1 ],
            [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1],
            [2, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1 ]
            ]
    # maze for R2 robot position as 1, P2 as 2, E2 as 3, D2 as 4, obstacle as 0 and all pssible moveable nodes as 1
    maze2 = [[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 0, 1, 1, 1],
            [1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1],
            [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1 ],
            [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ],
            [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1],
            [0, 1, 1, 0, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1 ]
            ]
    # maze for R3 robot position as 1, P3 as 2, E3 as 3, D3 as 4, obstacle as 0 and all pssible moveable nodes as 1
    maze3 = [[0, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1],
            [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1],
            [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1 ],
            [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1 ],
            [1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1],
            [1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1 ]
            ]
    
    astar_start_time = time.time() # start time for A*
    print('resultant path using A* algorithm')
    for i in range (3):
        if (i ==0):
            # R1 -> P1 -> E1 -> D1
            print('for robot R1') 
            # start and end location for R1 to P1
            start = (0, 0)     
            end = (5, 0)       
            path = astar(maze1, start, end)  # path from R1 to P1  
            print('R1 to P1 path ',path)
            # start and end location for P1 to E1
            start = (5, 0)    
            end = (0, 8)
            path = astar(maze1, start, end) # path from P1 to E1
            print('P1 to E1 path ',path)
            # start and end location for E1 to D1
            start = (0, 8)
            end = (3, 12)
            path = astar(maze1, start, end) # path from E1 to D1
            print('E1 to D1 path ',path,'\n')
            
        if (i ==1):
            # R2 -> P2 -> E2 -> D2
            print('for robot R2')
            # start and end location for R2 to P2
            start = (5, 9)
            end = (4, 6)
            path = astar(maze2, start, end) # path from R2 to P2
            print('R2 to P2 path ',path)
            # start and end location for P2 to E2
            start = (4, 6)
            end = (5, 5)
            path = astar(maze2, start, end) # path from P2 to E2
            print('P2 to E2 path ',path)
            # start and end location for E2 to D2
            start = (5, 5)
            end = (0, 10)
            path = astar(maze2, start, end)  # path from E2 to D2
            print('E2 to D2 path ',path, '\n')
            
        if (i ==2):
            # R3 -> P3 -> E3 -> D3
            print('for robot R3')
            # start and end location for R3 to P3
            start = (1, 7)
            end = (1, 12)
            path = astar(maze3, start, end) # path from R3 to P3
            print('R3 to P3 path ',path)
            # start and end location for P3 to E3
            start = (1, 12)
            end = (3, 15)
            path = astar(maze3, start, end) # path from P3 to E3
            print('P3 to E3 path ',path)
            # start and end location for E3 to D3
            start = (3, 15)
            end = (0, 5)
            path = astar(maze3, start, end) # path from E3 to D3
            print('E3 to D3 path ',path,'\n')
            
    astar_end_time = time.time() #end time for Astar
    astar_totaltime = astar_end_time - astar_start_time  # Total Time taken by A*
    
    #################      STEPS TO IMPLIMENT CONFLICT BASED SEARCH    #####################
    print('using path finding function')
    sp1 = [(0, 0), (4, 0), (0, 8), (3, 12)]  # path nodes of R1 as: [R1 location, P1 location, E1 location, D1 location] 
    sp2 = [(5, 11), (4, 6), (5, 5), (0, 10)]  # path nodes of R2 as: [R2 location, P2 location, E2 location, D2 location]
    sp3 = [(2, 7), (1, 12), (3, 15), (0, 5)]  # path nodes of R3 as: [R3 location, P3 location, E3 location, D3 location]
    
    cbc_start_time = time.time() # start time of conflict based search
   
    path1cbs, path2cbs, path3cbs = pathfinding_Robots(maze1, maze2, maze3, sp1, sp2, sp3) # getting CBS based paths
    print('resultant path using conflict based search algorithm \n')
    print('path for robot R1', path1cbs,'\n')
    print('path for robot R2', path2cbs,'\n')
    print('path for robot R3', path3cbs,'\n')
    
    cbc_end_time = time.time() # end time for conflict based search
    cbc_total = cbc_end_time - cbc_start_time # total time for CBS
    
    print('cbc total time', cbc_total,'\n A star total time', astar_totaltime,'\n')
    

if __name__ == '__main__':
    main()
    

resultant path using A* algorithm
for robot R1
R1 to P1 path  [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0)]
P1 to E1 path  [(5, 0), (5, 1), (5, 2), (4, 2), (4, 3), (4, 4), (4, 5), (3, 5), (3, 6), (2, 6), (2, 7), (2, 8), (1, 8), (0, 8)]
E1 to D1 path  [(0, 8), (0, 9), (0, 10), (1, 10), (1, 11), (2, 11), (2, 12), (3, 12)] 

for robot R2
R2 to P2 path  [(5, 9), (5, 8), (5, 7), (5, 6), (4, 6)]
P2 to E2 path  [(4, 6), (4, 5), (5, 5)]
E2 to D2 path  [(5, 5), (5, 6), (4, 6), (4, 7), (3, 7), (3, 8), (2, 8), (2, 9), (1, 9), (1, 10), (0, 10)] 

for robot R3
R3 to P3 path  [(1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12)]
P3 to E3 path  [(1, 12), (1, 13), (1, 14), (2, 14), (3, 14), (3, 15)]
E3 to D3 path  [(3, 15), (3, 14), (3, 13), (3, 12), (3, 11), (3, 10), (3, 9), (3, 8), (3, 7), (2, 7), (2, 6), (1, 6), (1, 5), (0, 5)] 

using path finding function
resultant path using conflict based search algorithm 

path for robot R1 [(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3), (4, 

## Test case 1

In [17]:
###########################  input of test case 1 (given in the figure)  #########################
#sp1 = [(0, 0), (5, 0), (0, 8), (3, 12)]  # path nodes of R1 as: [R1 location, P1 location, E1 location, D1 location] 
#sp2 = [(5, 9), (4, 6), (5, 5), (0, 10)]  # path nodes of R2 as: [R2 location, P2 location, E2 location, D2 location]
#sp3 = [(1, 7), (1, 12), (3, 15), (0, 5)]  # path nodes of R3 as: [R3 location, P3 location, E3 location, D3 location]   

## Test case 2

In [ ]:
#####################  input of test case 2 ####################################################
#sp1 = [(0, 0), (4, 0), (0, 8), (3, 12)]  # path nodes of R1 as: [R1 location, P1 location, E1 location, D1 location] 
#sp2 = [(5, 11), (4, 6), (5, 5), (0, 10)]  # path nodes of R2 as: [R2 location, P2 location, E2 location, D2 location]
#sp3 = [(1, 7), (1, 12), (3, 15), (0, 5)]  # path nodes of R3 as: [R3 location, P3 location, E3 location, D3 location]

## Test case 3

In [ ]:
#####################  input of test case 3 ####################################################
#sp1 = [(0, 0), (4, 0), (0, 8), (3, 12)]  # path nodes of R1 as: [R1 location, P1 location, E1 location, D1 location] 
#sp2 = [(5, 11), (4, 6), (5, 5), (0, 10)]  # path nodes of R2 as: [R2 location, P2 location, E2 location, D2 location]
#sp3 = [(2, 7), (1, 12), (3, 15), (0, 5)]  # path nodes of R3 as: [R3 location, P3 location, E3 location, D3 location]